# Prepare queries

In [13]:
import os
import pickle as p
import json

#from p_bloom_filter import build_bloom_filter
from phe import paillier
#from p_database import encode_data, encrypt_bloom_filter
from collections import defaultdict, namedtuple

data = None
SEQUENCE_TYPE = "public_addgene_full_sequences"
Gene = namedtuple("Gene", "name, pi, sequence, bloom")

In [16]:
f = 'sample.txt'
with open(f, 'r') as queries_file:
    text = queries_file.read()
    queries = text.splitlines()
queries_file.close()

In [18]:
out_loc = os.path.join('/local_data/atitus/data/addgene/queries')

In [20]:
id_ = 0
for query in queries:
    query_out = os.path.join(out_loc, 'query%s.fasta' % str(id_))
    f_out = open(query_out, 'w')
    f_out.write('> Query seq %s\n' % str(id_))
    f_out.write(str(query))
    f_out.close()
    id_ += 1

# Prepare database

In [9]:
file_loc = os.path.join('../', 'addgene-plasmids-sequences.json')
out_loc = os.path.join('/local_data/atitus/data/addgene/addgene_data')

In [15]:
def encode_data():

    with open('../addgene-plasmids-sequences.json') as data_file:
        raw_data = json.load(data_file)
    data_file.close()

    plasmids = raw_data['plasmids']
    id_ = 0
    # Read in and organize important fields of data from the raw data.
    for k in plasmids:
        name = k['name']
        if k['pi']:
            pi = k['pi'][0]
            if k['sequences'][SEQUENCE_TYPE]:
                sequence = k['sequences'][SEQUENCE_TYPE][0]
                
                query_out = os.path.join(out_loc, 'addgene%s.fasta' % str(id_))
                f_out = open(query_out, 'w')
                f_out.write('> Addgene seq %s\n' % str(id_))
                f_out.write(str(sequence))
                f_out.close()
                id_ += 1

In [14]:
encode_data()